In [1]:
import datetime
import numpy as np
import alpaca_trade_api as  tradeapi
import time
import talib as ta
import sys
sys.path.insert(0, '../')
import pandas as pd
from datetime import timedelta

In [2]:
import dtale
import plotly
import dash
import config

In [3]:
from config import binance_api_key, binance_secret_key
import cross_signal

In [4]:
import requests

In [5]:
#Python wrapper for binance
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager

In [6]:
client = Client(binance_api_key, binance_secret_key, tld='us')

In [7]:
depth = client.get_order_book(symbol='BNBBTC')

In [8]:
trades=client.get_recent_trades(symbol='ETHBUSD')

In [9]:
def binanceDataFrame(self, klines):
    df = pd.DataFrame(klines.reshape(-1,12),dtype=float, columns = ('Open Time',
                                                                    'Open',
                                                                    'High',
                                                                    'Low',
                                                                    'Close',
                                                                    'Volume',
                                                                    'Close time',
                                                                    'Quote asset volume',
                                                                    'Number of trades',
                                                                    'Taker buy base asset volume',
                                                                    'Taker buy quote asset volume',
                                                                    'Ignore'))

    df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')


    return df

In [10]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=4, price='High')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=4, price='Close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=4, price='Low')
        
        dataframe['dema_close'] = tab.DEMA(dataframe, timeperiod=4, price='Close')
        dataframe['sema_high'] = tab.EMA(dataframe, timeperiod=10, price='High')
        dataframe['sema_close'] = tab.EMA(dataframe, timeperiod=10, price='Close')
        dataframe['sema_low'] = tab.EMA(dataframe, timeperiod=10, price='Low')
        
        dataframe['ssema_close'] = tab.EMA(dataframe, timeperiod=35, price='Close')
        dataframe['sssema_close'] = tab.EMA(dataframe, timeperiod=40, price='Close')
        dataframe['ssssema_close'] = tab.EMA(dataframe, timeperiod=45, price='Close')
        dataframe['chaikosc'] = ta.ADOSC(dataframe['High'],dataframe['Low'],dataframe['Close'], np.asarray(dataframe['Quote asset volume'], dtype='float'))
        
        dataframe['mfi'] = ta.MFI(dataframe['High'],dataframe['Low'],dataframe['Close'], np.asarray(dataframe['Quote asset volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['High'],dataframe['Low'],dataframe['Close'])
        dataframe['obv'] = ta.OBV(dataframe['Close'], np.asarray(dataframe['Quote asset volume'], dtype='float'))
        
        dataframe['rsi'] = tab.RSI(dataframe['Close'], timeperiod=14)
        dataframe['adx'] = tab.ADX(dataframe['High'],dataframe['Low'],dataframe['Close'], timeperiod=14)
        dataframe['shortadx'] = tab.ADX(dataframe['High'],dataframe['Low'],dataframe['Close'], timeperiod=10)
        dataframe['srsi'] = tab.RSI(dataframe, timeperiod=3, price='Close')
       


        return dataframe

In [11]:
data=np.array(client.get_historical_klines('ETHBUSD', Client.KLINE_INTERVAL_5MINUTE, "14 May, 2021"))
df=binanceDataFrame(data,data)
df

,Open Time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2021-05-14 00:00:00,3711.73,3752.00,3696.81,3752.00,13.99823,1.620951e+12,52069.699867,37.0,11.57649,43046.018651,0.0
1,2021-05-14 00:05:00,3754.89,3803.52,3743.71,3800.30,10.86795,1.620951e+12,41038.239212,75.0,3.61144,13621.813235,0.0
2,2021-05-14 00:10:00,3802.38,3802.38,3775.10,3799.06,11.09068,1.620951e+12,42033.451671,71.0,5.01864,19025.329107,0.0
3,2021-05-14 00:15:00,3796.04,3796.04,3760.12,3787.67,9.90878,1.620952e+12,37471.155563,60.0,5.53258,20934.359302,0.0
4,2021-05-14 00:20:00,3781.61,3784.77,3773.86,3776.86,3.58612,1.620952e+12,13551.655659,27.0,3.01877,11405.983505,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1710,2021-05-19 22:30:00,2645.21,2674.91,2643.62,2668.26,23.80725,1.621464e+12,63281.115133,66.0,10.28926,27339.197168,0.0
1711,2021-05-19 22:35:00,2665.64,2690.57,2648.60,2658.16,50.81696,1.621464e+12,135326.506691,153.0,10.08428,26924.894850,0.0
1712,2021-05-19 22:40:00,2662.65,2702.54,2662.64,2687.54,65.35981,1.621464e+12,175274.727462,198.0,28.39463,76312.812392,0.0
1713,2021-05-19 22:45:00,2687.10,2719.73,2670.12,2709.04,53.02484,1.621465e+12,142640.954091,144.0,12.82959,34630.668087,0.0


In [12]:
df=populateindicators(df)

In [13]:
dtale.show(df)

In [14]:
data3=np.array(client.get_historical_klines('ETHBUSD', Client.KLINE_INTERVAL_30MINUTE, "14 May, 2021"))
df3=binanceDataFrame(data3,data3)
df3=populateindicators(df3)
dtale.show(df3)